<a href="https://colab.research.google.com/github/jeongname/jeongname/blob/main/220407_hj_SANUP_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리, 패키지

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install transformers

     |████████████████████████████████| 325 kB 5.3 MB/s 
     |████████████████████████████████| 4.0 MB 70.0 MB/s 
     |████████████████████████████████| 136 kB 84.2 MB/s 
     |████████████████████████████████| 77 kB 7.6 MB/s 
     |████████████████████████████████| 1.1 MB 67.8 MB/s 
     |████████████████████████████████| 212 kB 71.1 MB/s 
     |████████████████████████████████| 127 kB 75.8 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 271 kB 79.2 MB/s 
     |████████████████████████████████| 144 kB 77.6 MB/s 
     |████████████████████████████████| 895 kB 74.2 MB/s 
     |████████████████████████████████| 596 kB 77.0 MB/s 
     |████████████████████████████████| 6.5 MB 61.3 MB/s 
     |████████████████████████████████| 1.2 MB 61.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
   

- TPU 사용을 위한 accelerate

In [3]:
from accelerate import Accelerator

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from transformers import TrainingArguments, Trainer
from transformers import AutoModel,AutoModelForSequenceClassification, AutoConfig, AutoTokenizer
import gc
from transformers import AdamW
from transformers import get_scheduler, get_cosine_with_hard_restarts_schedule_with_warmup
from tqdm.auto import tqdm
from datasets import load_metric
import warnings
warnings.filterwarnings('ignore')
from accelerate import notebook_launcher
from pathlib import Path
from sklearn.model_selection import train_test_split

In [5]:
df = pd.read_csv('/content/drive/MyDrive/SANUP/1. 실습용자료.txt', sep='|', encoding='cp949')
test = pd.read_csv('/content/drive/MyDrive/SANUP/2. 모델개발용자료.txt', sep='|', encoding='cp949')
submission = pd.read_csv('/content/drive/MyDrive/SANUP/답안 작성용 파일.csv', encoding='cp949')

In [6]:
df = df.fillna('')

In [7]:
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육
...,...,...,...,...,...,...,...
999995,id_0999996,C,13,134,제품입고,워싱,청바지워싱
999996,id_0999997,F,42,424,현장에서,고객의요청에의해,실내인테리어
999997,id_0999998,G,47,474,영업점에서,일반소비자에게,여성의류 판매
999998,id_0999999,P,85,856,사업장에서,일반고객을대상으로,필라테스


In [8]:
df = df.sample(n=100000)
df = df.reset_index(drop=True)

In [9]:
df_train = df[:90000]
df_test_1 = df[90000:]
df_test = df_test_1

In [10]:
df_test['text'] = df_test['text_obj'].astype(str) + " " + df_test['text_mthd'].astype(str) + " " + df_test['text_deal'].astype(str)
df_test = df_test[['text']]
df_test = df_test.reset_index(drop=True)

In [11]:
df_test

,text
0,"철 프레스가공,용접 자동차부품(외관)"
1,커피숍에서 접객시설을 갖추고 커피 제공
2,"영업장에서 접객시설을 갖추고 치킨,호프"
3,어린이집에서 군청의인가를받아 영유아보육
4,영업장에서 고객대상으로 화장품소매
...,...
9995,매장에서 접객시설을갖추고 칼국수
9996,개인택시로 일반인을 대상으로 승객운송서비스
9997,사업장에서 노니직수입 온라인판매
9998,"미장원에서 두발미용서비스제공 파마,컷트"


### digit_1 예측하기

In [12]:
df_digit1 = df_train

In [13]:
df_digit1['text'] = df_digit1['text_obj'].astype(str) + " " + df_digit1['text_mthd'].astype(str) + " " + df_digit1['text_deal'].astype(str)
df_digit1['target'] = df_digit1['digit_1']
df_digit1

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text,target
0,id_0950013,S,96,961,미장원에서,일반고객대상으로,"컷트, 파마","미장원에서 일반고객대상으로 컷트, 파마",S
1,id_0184598,F,42,426,남의현장에서,고객의요구에따라라,건설장비임대,남의현장에서 고객의요구에따라라 건설장비임대,F
2,id_0766405,G,47,472,정육점에서,소비자에게,"소,돼지고기판매","정육점에서 소비자에게 소,돼지고기판매",G
3,id_0046046,L,68,682,아파트관리소에서,입주민을 위해,아파트 관리 서비스,아파트관리소에서 입주민을 위해 아파트 관리 서비스,L
4,id_0267644,H,49,493,개별화물자동차로,고객의 요청으로,화물운송서비스,개별화물자동차로 고객의 요청으로 화물운송서비스,H
...,...,...,...,...,...,...,...,...,...
89995,id_0503799,L,68,682,관리사무실,농공아파트를 위하여,아파트 관리,관리사무실 농공아파트를 위하여 아파트 관리,L
89996,id_0306020,I,55,559,고시텔에서,일반인을 대상으로,고시텔운영,고시텔에서 일반인을 대상으로 고시텔운영,I
89997,id_0819988,I,55,551,호텔에서,객실을 이용,룸서비스 제공,호텔에서 객실을 이용 룸서비스 제공,I
89998,id_0116261,C,22,222,아크릴,CNC가공,핸드폰 케이스,아크릴 CNC가공 핸드폰 케이스,C


In [14]:
train_digit1 = df_digit1[['text', 'target']]
train_digit1

,text,target
0,"미장원에서 일반고객대상으로 컷트, 파마",S
1,남의현장에서 고객의요구에따라라 건설장비임대,F
2,"정육점에서 소비자에게 소,돼지고기판매",G
3,아파트관리소에서 입주민을 위해 아파트 관리 서비스,L
4,개별화물자동차로 고객의 요청으로 화물운송서비스,H
...,...,...
89995,관리사무실 농공아파트를 위하여 아파트 관리,L
89996,고시텔에서 일반인을 대상으로 고시텔운영,I
89997,호텔에서 객실을 이용 룸서비스 제공,I
89998,아크릴 CNC가공 핸드폰 케이스,C


### 1. digit_2 예측하기

In [15]:
df_digit2 = df_train

In [16]:
df_digit2['text'] = df_digit2['text_obj'].astype(str) + " " + df_digit2['text_mthd'].astype(str) + " " + df_digit2['text_deal'].astype(str)
df_digit2['target'] = df_digit2['digit_2'].astype(str)
df_digit2

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text,target
0,id_0950013,S,96,961,미장원에서,일반고객대상으로,"컷트, 파마","미장원에서 일반고객대상으로 컷트, 파마",96
1,id_0184598,F,42,426,남의현장에서,고객의요구에따라라,건설장비임대,남의현장에서 고객의요구에따라라 건설장비임대,42
2,id_0766405,G,47,472,정육점에서,소비자에게,"소,돼지고기판매","정육점에서 소비자에게 소,돼지고기판매",47
3,id_0046046,L,68,682,아파트관리소에서,입주민을 위해,아파트 관리 서비스,아파트관리소에서 입주민을 위해 아파트 관리 서비스,68
4,id_0267644,H,49,493,개별화물자동차로,고객의 요청으로,화물운송서비스,개별화물자동차로 고객의 요청으로 화물운송서비스,49
...,...,...,...,...,...,...,...,...,...
89995,id_0503799,L,68,682,관리사무실,농공아파트를 위하여,아파트 관리,관리사무실 농공아파트를 위하여 아파트 관리,68
89996,id_0306020,I,55,559,고시텔에서,일반인을 대상으로,고시텔운영,고시텔에서 일반인을 대상으로 고시텔운영,55
89997,id_0819988,I,55,551,호텔에서,객실을 이용,룸서비스 제공,호텔에서 객실을 이용 룸서비스 제공,55
89998,id_0116261,C,22,222,아크릴,CNC가공,핸드폰 케이스,아크릴 CNC가공 핸드폰 케이스,22


In [17]:
train_digit2 = df_digit2[['text', 'target']]
train_digit2

,text,target
0,"미장원에서 일반고객대상으로 컷트, 파마",96
1,남의현장에서 고객의요구에따라라 건설장비임대,42
2,"정육점에서 소비자에게 소,돼지고기판매",47
3,아파트관리소에서 입주민을 위해 아파트 관리 서비스,68
4,개별화물자동차로 고객의 요청으로 화물운송서비스,49
...,...,...
89995,관리사무실 농공아파트를 위하여 아파트 관리,68
89996,고시텔에서 일반인을 대상으로 고시텔운영,55
89997,호텔에서 객실을 이용 룸서비스 제공,55
89998,아크릴 CNC가공 핸드폰 케이스,22


### digit_3 예측하기

In [18]:
df_digit3 = df_train

In [19]:
df_digit3['text'] = df_digit3['text_obj'].astype(str) + " " + df_digit3['text_mthd'].astype(str) + " " + df_digit3['text_deal'].astype(str)
df_digit3['target'] = df_digit3['digit_3'].astype(str)
df_digit3

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text,target
0,id_0950013,S,96,961,미장원에서,일반고객대상으로,"컷트, 파마","미장원에서 일반고객대상으로 컷트, 파마",961
1,id_0184598,F,42,426,남의현장에서,고객의요구에따라라,건설장비임대,남의현장에서 고객의요구에따라라 건설장비임대,426
2,id_0766405,G,47,472,정육점에서,소비자에게,"소,돼지고기판매","정육점에서 소비자에게 소,돼지고기판매",472
3,id_0046046,L,68,682,아파트관리소에서,입주민을 위해,아파트 관리 서비스,아파트관리소에서 입주민을 위해 아파트 관리 서비스,682
4,id_0267644,H,49,493,개별화물자동차로,고객의 요청으로,화물운송서비스,개별화물자동차로 고객의 요청으로 화물운송서비스,493
...,...,...,...,...,...,...,...,...,...
89995,id_0503799,L,68,682,관리사무실,농공아파트를 위하여,아파트 관리,관리사무실 농공아파트를 위하여 아파트 관리,682
89996,id_0306020,I,55,559,고시텔에서,일반인을 대상으로,고시텔운영,고시텔에서 일반인을 대상으로 고시텔운영,559
89997,id_0819988,I,55,551,호텔에서,객실을 이용,룸서비스 제공,호텔에서 객실을 이용 룸서비스 제공,551
89998,id_0116261,C,22,222,아크릴,CNC가공,핸드폰 케이스,아크릴 CNC가공 핸드폰 케이스,222


In [20]:
train_digit3 = df_digit3[['text', 'target']]
train_digit3

,text,target
0,"미장원에서 일반고객대상으로 컷트, 파마",961
1,남의현장에서 고객의요구에따라라 건설장비임대,426
2,"정육점에서 소비자에게 소,돼지고기판매",472
3,아파트관리소에서 입주민을 위해 아파트 관리 서비스,682
4,개별화물자동차로 고객의 요청으로 화물운송서비스,493
...,...,...
89995,관리사무실 농공아파트를 위하여 아파트 관리,682
89996,고시텔에서 일반인을 대상으로 고시텔운영,559
89997,호텔에서 객실을 이용 룸서비스 제공,551
89998,아크릴 CNC가공 핸드폰 케이스,222


# 토크나이저, 함수

In [21]:
from transformers import AutoTokenizer
from transformers import ElectraModel, ElectraTokenizer

checkpoint = "monologg/koelectra-base-v3-discriminator"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/257k [00:00<?, ?B/s]

In [22]:
class BERTDataset(torch.utils.data.Dataset):
    def __init__(self, pair_dataset, label):
        self.pair_dataset = pair_dataset
        self.label = label

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
        item['labels'] = torch.tensor(self.label[idx])
        
        return item

    def __len__(self):
        return len(self.label)

### digit_1

In [23]:
my_dict1 = {}
for i in range(len(train_digit1['target'].unique())):
  my_dict1[train_digit1['target'].unique()[i]] = i

def target_to_num_digit1(target:str):
  return my_dict1[target]

train_digit1['target'] = train_digit1['target'].map(target_to_num_digit1)
train_digit1

,text,target
0,"미장원에서 일반고객대상으로 컷트, 파마",0
1,남의현장에서 고객의요구에따라라 건설장비임대,1
2,"정육점에서 소비자에게 소,돼지고기판매",2
3,아파트관리소에서 입주민을 위해 아파트 관리 서비스,3
4,개별화물자동차로 고객의 요청으로 화물운송서비스,4
...,...,...
89995,관리사무실 농공아파트를 위하여 아파트 관리,3
89996,고시텔에서 일반인을 대상으로 고시텔운영,6
89997,호텔에서 객실을 이용 룸서비스 제공,6
89998,아크릴 CNC가공 핸드폰 케이스,11


In [24]:
my_dict1[''] = len(my_dict1)

In [25]:
train_digit1[train_digit1['text'].isnull()]

,text,target


### digit_2

In [26]:
my_dict2 = {}
for i in range(len(train_digit2['target'].unique())):
  my_dict2[train_digit2['target'].unique()[i]] = i

def target_to_num_digit2(target:str):
  return my_dict2[target]

train_digit2['target'] = train_digit2['target'].map(target_to_num_digit2)
train_digit2

,text,target
0,"미장원에서 일반고객대상으로 컷트, 파마",0
1,남의현장에서 고객의요구에따라라 건설장비임대,1
2,"정육점에서 소비자에게 소,돼지고기판매",2
3,아파트관리소에서 입주민을 위해 아파트 관리 서비스,3
4,개별화물자동차로 고객의 요청으로 화물운송서비스,4
...,...,...
89995,관리사무실 농공아파트를 위하여 아파트 관리,3
89996,고시텔에서 일반인을 대상으로 고시텔운영,13
89997,호텔에서 객실을 이용 룸서비스 제공,13
89998,아크릴 CNC가공 핸드폰 케이스,28


In [27]:
my_dict2[''] = len(my_dict2)

In [28]:
train_digit2[train_digit2['text'].isnull()]

,text,target


### digit_3

In [29]:
my_dict3 = {}
for i in range(len(train_digit3['target'].unique())):
  my_dict3[train_digit3['target'].unique()[i]] = i

def target_to_num_digit3(target:str):
  return my_dict3[target]

train_digit3['target'] = train_digit3['target'].map(target_to_num_digit3)
train_digit3

,text,target
0,"미장원에서 일반고객대상으로 컷트, 파마",0
1,남의현장에서 고객의요구에따라라 건설장비임대,1
2,"정육점에서 소비자에게 소,돼지고기판매",2
3,아파트관리소에서 입주민을 위해 아파트 관리 서비스,3
4,개별화물자동차로 고객의 요청으로 화물운송서비스,4
...,...,...
89995,관리사무실 농공아파트를 위하여 아파트 관리,3
89996,고시텔에서 일반인을 대상으로 고시텔운영,106
89997,호텔에서 객실을 이용 룸서비스 제공,15
89998,아크릴 CNC가공 핸드폰 케이스,47


In [30]:
my_dict3[''] = len(my_dict3)

In [31]:
train_digit3[train_digit3['text'].isnull()]

,text,target


In [32]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [33]:
display(train_digit1)
display(train_digit2)
display(train_digit3)

,text,target
0,"미장원에서 일반고객대상으로 컷트, 파마",0
1,남의현장에서 고객의요구에따라라 건설장비임대,1
2,"정육점에서 소비자에게 소,돼지고기판매",2
3,아파트관리소에서 입주민을 위해 아파트 관리 서비스,3
4,개별화물자동차로 고객의 요청으로 화물운송서비스,4
...,...,...
89995,관리사무실 농공아파트를 위하여 아파트 관리,3
89996,고시텔에서 일반인을 대상으로 고시텔운영,6
89997,호텔에서 객실을 이용 룸서비스 제공,6
89998,아크릴 CNC가공 핸드폰 케이스,11


,text,target
0,"미장원에서 일반고객대상으로 컷트, 파마",0
1,남의현장에서 고객의요구에따라라 건설장비임대,1
2,"정육점에서 소비자에게 소,돼지고기판매",2
3,아파트관리소에서 입주민을 위해 아파트 관리 서비스,3
4,개별화물자동차로 고객의 요청으로 화물운송서비스,4
...,...,...
89995,관리사무실 농공아파트를 위하여 아파트 관리,3
89996,고시텔에서 일반인을 대상으로 고시텔운영,13
89997,호텔에서 객실을 이용 룸서비스 제공,13
89998,아크릴 CNC가공 핸드폰 케이스,28


,text,target
0,"미장원에서 일반고객대상으로 컷트, 파마",0
1,남의현장에서 고객의요구에따라라 건설장비임대,1
2,"정육점에서 소비자에게 소,돼지고기판매",2
3,아파트관리소에서 입주민을 위해 아파트 관리 서비스,3
4,개별화물자동차로 고객의 요청으로 화물운송서비스,4
...,...,...
89995,관리사무실 농공아파트를 위하여 아파트 관리,3
89996,고시텔에서 일반인을 대상으로 고시텔운영,106
89997,호텔에서 객실을 이용 룸서비스 제공,15
89998,아크릴 CNC가공 핸드폰 케이스,47


# 학습

* Model
  + Roberta-large

* Optimizer
  + AdamW

* Learning rate scheduler
  + Cosine annealing with warmup

### digit_1

In [34]:
def training_function():
  accelerator = Accelerator()

  train_dataloader = torch.utils.data.DataLoader(
                    train_dataset, 
                    batch_size=32, sampler=train_subsampler)
  eval_dataloader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size=32, sampler=test_subsampler)

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = len(train_digit1['target'].unique())
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

  optimizer = AdamW(model.parameters(), lr=	1e-5)

  model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

  num_epochs = 10
  num_training_steps = num_epochs * len(train_dataloader)
  progress_bar = tqdm(range(num_training_steps))
  lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
      optimizer=optimizer,
      num_warmup_steps=1,
      num_training_steps=num_training_steps,
  )
  for epoch in range(num_epochs):
      train_acc = 0.0
      test_acc = 0.0

      model.train()
      for batch_id, batch in enumerate(train_dataloader):
          outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
          loss = F.cross_entropy(outputs[0], batch['labels'])
          accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)
          train_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

      model.eval()
      for batch_id, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])

        test_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} test acc {}".format(epoch+1, test_acc / (batch_id+1)))
      gc.collect()
  accelerator.wait_for_everyone()
  unwrapped_model = accelerator.unwrap_model(model)
  unwrapped_model.save_pretrained('/content/drive/MyDrive/220406_inference_digit1/model' + str(fold), save_function=accelerator.save)

In [36]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print('--------------------------------')

tokenized_train = tokenizer(
  list(train_digit1['text']),
  return_tensors="pt",
  max_length=64, # Max_Length = 64
  padding=True,
  truncation=True,
  add_special_tokens=True
  )
  
for fold, (train_ids, test_ids) in enumerate(kfold.split(train_digit1, train_digit1['target'])):
  my_file = Path("/content/drive/MyDrive/220406_inference_digit1/model"+str(fold))
  if my_file.is_dir():
    continue

  print(f'FOLD {fold}')

  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

  train_label = train_digit1['target']

  train_dataset = BERTDataset(tokenized_train, train_label)

  notebook_launcher(training_function)

ValueError: ignored

In [37]:
train_digit1

,text,target
0,"미장원에서 일반고객대상으로 컷트, 파마",0
1,남의현장에서 고객의요구에따라라 건설장비임대,1
2,"정육점에서 소비자에게 소,돼지고기판매",2
3,아파트관리소에서 입주민을 위해 아파트 관리 서비스,3
4,개별화물자동차로 고객의 요청으로 화물운송서비스,4
...,...,...
89995,관리사무실 농공아파트를 위하여 아파트 관리,3
89996,고시텔에서 일반인을 대상으로 고시텔운영,6
89997,호텔에서 객실을 이용 룸서비스 제공,6
89998,아크릴 CNC가공 핸드폰 케이스,11


### digit_2

In [38]:
def training_function():
  accelerator = Accelerator()

  train_dataloader = torch.utils.data.DataLoader(
                    train_dataset, 
                    batch_size=32, sampler=train_subsampler)
  eval_dataloader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size=32, sampler=test_subsampler)

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = len(train_digit2['target'].unique())
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

  optimizer = AdamW(model.parameters(), lr=	1e-5)

  model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

  num_epochs = 10
  num_training_steps = num_epochs * len(train_dataloader)
  progress_bar = tqdm(range(num_training_steps))
  lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
      optimizer=optimizer,
      num_warmup_steps=1,
      num_training_steps=num_training_steps,
  )
  for epoch in range(num_epochs):
      train_acc = 0.0
      test_acc = 0.0

      model.train()
      for batch_id, batch in enumerate(train_dataloader):
          outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
          loss = F.cross_entropy(outputs[0], batch['labels'])
          accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)
          train_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

      model.eval()
      for batch_id, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])

        test_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} test acc {}".format(epoch+1, test_acc / (batch_id+1)))
      gc.collect()
  accelerator.wait_for_everyone()
  unwrapped_model = accelerator.unwrap_model(model)
  unwrapped_model.save_pretrained('/content/drive/MyDrive/220406_inference_digit2/model' + str(fold), save_function=accelerator.save)

In [39]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print('--------------------------------')

tokenized_train = tokenizer(
  list(train_digit2['text']),
  return_tensors="pt",
  max_length=64, # Max_Length = 64
  padding=True,
  truncation=True,
  add_special_tokens=True
  )
  
for fold, (train_ids, test_ids) in enumerate(kfold.split(train_digit2, train_digit2['target'])):
  my_file = Path("/content/drive/MyDrive/220406_inference_digit2/model"+str(fold))
  
  if fold == 1:
    break

  if my_file.is_dir():
    continue

  print(f'FOLD {fold}')

  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

  train_label = train_digit2['target']

  train_dataset = BERTDataset(tokenized_train, train_label)

  notebook_launcher(training_function)

--------------------------------
FOLD 0
Launching a training on 8 TPU cores.


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

  0%|          | 0/2820 [00:00<?, ?it/s]

  0%|          | 0/2820 [00:00<?, ?it/s]

  0%|          | 0/2820 [00:00<?, ?it/s]

...

  0%|          | 0/2820 [00:00<?, ?it/s]

  0%|          | 0/2820 [00:00<?, ?it/s]

  0%|          | 0/2820 [00:00<?, ?it/s]

  0%|          | 0/2820 [00:00<?, ?it/s]

epoch 1 train acc 0.41600177304964536
epoch 1 train acc 0.4229831560283688
epoch 1 train acc 0.415447695035461
epoch 1 train acc 0.4186613475177305
epoch 1 train acc 0.42320478723404253
epoch 1 train acc 0.4162234042553192
epoch 1 train acc 0.41688829787234044
epoch 1 train acc 0.4208776595744681
epoch 1 test acc 0.6795774647887324
epoch 1 test acc 0.6720950704225352
epoch 1 test acc 0.6659330985915493
epoch 1 test acc 0.6725352112676056
epoch 1 test acc 0.6685739436619719
epoch 1 test acc 0.6690140845070423
epoch 1 test acc 0.6773767605633803
epoch 1 test acc 0.6764964788732394
epoch 2 train acc 0.7338209219858156
epoch 2 train acc 0.7358156028368794
epoch 2 train acc 0.7296099290780141
epoch 2 train acc 0.7195257092198581
epoch 2 train acc 0.7236258865248227
epoch 2 train acc 0.727061170212766
epoch 2 train acc 0.7386968085106383
epoch 2 train acc 0.7248448581560284
epoch 2 test acc 0.8023767605633803
epoch 2 test acc 0.7900528169014085
epoch 2 test acc 0.7825704225352113
epoch 2 tes

### digit_3

In [40]:
def training_function():
  accelerator = Accelerator()

  train_dataloader = torch.utils.data.DataLoader(
                    train_dataset, 
                    batch_size=32, sampler=train_subsampler)
  eval_dataloader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size=32, sampler=test_subsampler)

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = len(train_digit3['target'].unique())
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

  optimizer = AdamW(model.parameters(), lr=	1e-5)

  model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

  num_epochs = 10
  num_training_steps = num_epochs * len(train_dataloader)
  progress_bar = tqdm(range(num_training_steps))
  lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
      optimizer=optimizer,
      num_warmup_steps=1,
      num_training_steps=num_training_steps,
  )
  for epoch in range(num_epochs):
      train_acc = 0.0
      test_acc = 0.0

      model.train()
      for batch_id, batch in enumerate(train_dataloader):
          outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
          loss = F.cross_entropy(outputs[0], batch['labels'])
          accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)
          train_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

      model.eval()
      for batch_id, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])

        test_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} test acc {}".format(epoch+1, test_acc / (batch_id+1)))
      gc.collect()
  accelerator.wait_for_everyone()
  unwrapped_model = accelerator.unwrap_model(model)
  unwrapped_model.save_pretrained('/content/drive/MyDrive/220406_inference_digit3/model' + str(fold), save_function=accelerator.save)

In [42]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print('--------------------------------')

tokenized_train = tokenizer(
  list(train_digit3['text']),
  return_tensors="pt",
  max_length=64, # Max_Length = 64
  padding=True,
  truncation=True,
  add_special_tokens=True
  )
  
for fold, (train_ids, test_ids) in enumerate(kfold.split(train_digit3, train_digit3['target'])):
  my_file = Path("/content/drive/MyDrive/220406_inference_digit3/model"+str(fold))

  if fold == 1:
    break

  if my_file.is_dir():
    continue

  print(f'FOLD {fold}')

  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

  train_label = train_digit3['target']

  train_dataset = BERTDataset(tokenized_train, train_label)

  notebook_launcher(training_function)

--------------------------------
FOLD 0
Launching a training on 8 TPU cores.


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

  0%|          | 0/2820 [00:00<?, ?it/s]

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

  0%|          | 0/2820 [00:00<?, ?it/s]

...

  0%|          | 0/2820 [00:00<?, ?it/s]

  0%|          | 0/2820 [00:00<?, ?it/s]

  0%|          | 0/2820 [00:00<?, ?it/s]

  0%|          | 0/2820 [00:00<?, ?it/s]

epoch 1 train acc 0.20866578014184398
epoch 1 train acc 0.1983599290780142
epoch 1 train acc 0.19614361702127658
epoch 1 train acc 0.20234929078014185
epoch 1 train acc 0.20345744680851063
epoch 1 train acc 0.19680851063829788
epoch 1 train acc 0.2042331560283688
epoch 1 train acc 0.19703014184397163
epoch 1 test acc 0.3587147887323944
epoch 1 test acc 0.3485915492957746
epoch 1 test acc 0.3587147887323944
epoch 1 test acc 0.3261443661971831
epoch 1 test acc 0.3494718309859155
epoch 1 test acc 0.3516725352112676
epoch 1 test acc 0.3512323943661972
epoch 1 test acc 0.3459507042253521
epoch 2 train acc 0.46531471631205673
epoch 2 train acc 0.4605496453900709
epoch 2 train acc 0.45722517730496454
epoch 2 train acc 0.4542331560283688
epoch 2 train acc 0.4560062056737589
epoch 2 train acc 0.45933067375886527
epoch 2 train acc 0.4616578014184397
epoch 2 train acc 0.4521276595744681
epoch 2 test acc 0.5704225352112676
epoch 2 test acc 0.5985915492957746
epoch 2 test acc 0.585387323943662
epoc

In [43]:
train_digit3['target'].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

# 추론

### digit_1

In [44]:
df_test_digit1 = df_test
df_test_digit1

,text
0,"철 프레스가공,용접 자동차부품(외관)"
1,커피숍에서 접객시설을 갖추고 커피 제공
2,"영업장에서 접객시설을 갖추고 치킨,호프"
3,어린이집에서 군청의인가를받아 영유아보육
4,영업장에서 고객대상으로 화장품소매
...,...
9995,매장에서 접객시설을갖추고 칼국수
9996,개인택시로 일반인을 대상으로 승객운송서비스
9997,사업장에서 노니직수입 온라인판매
9998,"미장원에서 두발미용서비스제공 파마,컷트"


In [45]:
df_test_digit1['target'] = ''

In [46]:
df_test_digit1

,text,target
0,"철 프레스가공,용접 자동차부품(외관)",
1,커피숍에서 접객시설을 갖추고 커피 제공,
2,"영업장에서 접객시설을 갖추고 치킨,호프",
3,어린이집에서 군청의인가를받아 영유아보육,
4,영업장에서 고객대상으로 화장품소매,
...,...,...
9995,매장에서 접객시설을갖추고 칼국수,
9996,개인택시로 일반인을 대상으로 승객운송서비스,
9997,사업장에서 노니직수입 온라인판매,
9998,"미장원에서 두발미용서비스제공 파마,컷트",


In [51]:
train_digit1['target'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18])

In [49]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

test_label = df_test_digit1['target'].map(target_to_num_digit1)

tokenized_test = tokenizer(
    list(df_test_digit1['text']),
    return_tensors="pt",
    max_length=64,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

test_dataset = BERTDataset(tokenized_test, test_label)

dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

for fold in range(5):

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = 225
  model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/220406_inference_digit1/model' + str(fold), num_labels=len(train_digit1['target'].unique()))
  model.resize_token_embeddings(tokenizer.vocab_size)
  accelerator = Accelerator()
  model = accelerator.unwrap_model(model)

  output_pred = []
  output_prob = []

  model, dataloader= accelerator.prepare(model, dataloader)

  model.eval()

  for i, data in enumerate(tqdm(dataloader)):
      with torch.no_grad():
          outputs = model(
              input_ids=data['input_ids'],
              attention_mask=data['attention_mask']
          )
      logits = outputs[0]
      prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
      logits = logits.detach().cpu().numpy()
      result = np.argmax(logits, axis=-1)
      output_pred.append(result)
      output_prob.append(prob)
      
  pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()

  def num_to_label(label):
      label_dict = {v: k for k, v in my_dict1.items()}
      str_label = []

      for i, v in enumerate(label):
          str_label.append([i,label_dict[v]])
      
      return str_label

  answer = num_to_label(pred_answer)

  df_label1 = pd.DataFrame(answer, columns=['index', 'label'])
  df_prob1 = pd.DataFrame(output_prob)

  df_label1.to_csv('/content/drive/MyDrive/220406_inference_digit1/pred_label'+str(fold)+'.csv', index=False)
  df_prob1.to_csv('/content/drive/MyDrive/220406_inference_digit1/pred_prob'+str(fold)+'.csv', index=False)

  0%|          | 0/313 [00:00<?, ?it/s]

OSError: ignored

In [52]:
df_label1

,index,label
0,0,C
1,1,I
2,2,I
3,3,Q
4,4,G
...,...,...
9995,9995,I
9996,9996,H
9997,9997,G
9998,9998,S


### digit_2

In [53]:
df_test_digit2 = df_test
df_test_digit2

,text,target
0,"철 프레스가공,용접 자동차부품(외관)",
1,커피숍에서 접객시설을 갖추고 커피 제공,
2,"영업장에서 접객시설을 갖추고 치킨,호프",
3,어린이집에서 군청의인가를받아 영유아보육,
4,영업장에서 고객대상으로 화장품소매,
...,...,...
9995,매장에서 접객시설을갖추고 칼국수,
9996,개인택시로 일반인을 대상으로 승객운송서비스,
9997,사업장에서 노니직수입 온라인판매,
9998,"미장원에서 두발미용서비스제공 파마,컷트",


In [54]:
df_test_digit2['target'] = ''

In [55]:
train_digit2['target'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70])

In [56]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

test_label = df_test_digit2['target'].map(target_to_num_digit2)

tokenized_test = tokenizer(
    list(df_test_digit2['text']),
    return_tensors="pt",
    max_length=64,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

test_dataset = BERTDataset(tokenized_test, test_label)

dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

for fold in range(5):

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = 225
  model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/220406_inference_digit2/model' + str(fold), num_labels=len(train_digit2['target'].unique()))
  model.resize_token_embeddings(tokenizer.vocab_size)
  accelerator = Accelerator()
  model = accelerator.unwrap_model(model)

  output_pred = []
  output_prob = []

  model, dataloader= accelerator.prepare(model, dataloader)

  model.eval()

  for i, data in enumerate(tqdm(dataloader)):
      with torch.no_grad():
          outputs = model(
              input_ids=data['input_ids'],
              attention_mask=data['attention_mask']
          )
      logits = outputs[0]
      prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
      logits = logits.detach().cpu().numpy()
      result = np.argmax(logits, axis=-1)
      output_pred.append(result)
      output_prob.append(prob)
      
  pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()

  def num_to_label(label):
      label_dict = {v: k for k, v in my_dict2.items()}
      str_label = []

      for i, v in enumerate(label):
          str_label.append([i,label_dict[v]])
      
      return str_label

  answer = num_to_label(pred_answer)

  df_label2 = pd.DataFrame(answer, columns=['index', 'label'])
  df_prob2 = pd.DataFrame(output_prob)

  df_label2.to_csv('/content/drive/MyDrive/220406_inference_digit2/pred_label'+str(fold)+'.csv', index=False)
  df_prob2.to_csv('/content/drive/MyDrive/220406_inference_digit2/pred_prob'+str(fold)+'.csv', index=False)

  0%|          | 0/313 [00:00<?, ?it/s]

OSError: ignored

In [58]:
df_label2['label'].unique()

array(['25', '56', '87', '47', '58', '49', '75', '14', '73', '68', '46',
       '94', '91', '55', '72', '85', '95', '86', '96', '10', '71', '18',
       '13', '42', '22', '45', '64', '41', '29', '52', '76', '90', '84',
       '65', '34'], dtype=object)

### digit3 추론

In [59]:
df_test_digit3 = df_test
df_test_digit3

,text,target
0,"철 프레스가공,용접 자동차부품(외관)",
1,커피숍에서 접객시설을 갖추고 커피 제공,
2,"영업장에서 접객시설을 갖추고 치킨,호프",
3,어린이집에서 군청의인가를받아 영유아보육,
4,영업장에서 고객대상으로 화장품소매,
...,...,...
9995,매장에서 접객시설을갖추고 칼국수,
9996,개인택시로 일반인을 대상으로 승객운송서비스,
9997,사업장에서 노니직수입 온라인판매,
9998,"미장원에서 두발미용서비스제공 파마,컷트",


In [60]:
df_test_digit3['target'] = ''

In [61]:
train_digit3['target'].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [62]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

test_label = df_test_digit3['target'].map(target_to_num_digit3)

tokenized_test = tokenizer(
    list(df_test_digit3['text']),
    return_tensors="pt",
    max_length=64,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

test_dataset = BERTDataset(tokenized_test, test_label)

dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

for fold in range(5):

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = 225
  model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/220406_inference_digit3/model' + str(fold), num_labels=len(train_digit3['target'].unique()))
  model.resize_token_embeddings(tokenizer.vocab_size)
  accelerator = Accelerator()
  model = accelerator.unwrap_model(model)

  output_pred = []
  output_prob = []

  model, dataloader= accelerator.prepare(model, dataloader)

  model.eval()

  for i, data in enumerate(tqdm(dataloader)):
      with torch.no_grad():
          outputs = model(
              input_ids=data['input_ids'],
              attention_mask=data['attention_mask']
          )
      logits = outputs[0]
      prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
      logits = logits.detach().cpu().numpy()
      result = np.argmax(logits, axis=-1)
      output_pred.append(result)
      output_prob.append(prob)
      
  pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()

  def num_to_label(label):
      label_dict = {v: k for k, v in my_dict3.items()}
      str_label = []

      for i, v in enumerate(label):
          str_label.append([i,label_dict[v]])
      
      return str_label

  answer = num_to_label(pred_answer)

  df_label3 = pd.DataFrame(answer, columns=['index', 'label'])
  df_prob3 = pd.DataFrame(output_prob)

  df_label3.to_csv('/content/drive/MyDrive/220406_inference_digit3/pred_label'+str(fold)+'.csv', index=False)
  df_prob3.to_csv('/content/drive/MyDrive/220406_inference_digit3/pred_prob'+str(fold)+'.csv', index=False)

  0%|          | 0/313 [00:00<?, ?it/s]

OSError: ignored

In [64]:
df_label3['label'].unique()

array(['259', '562', '561', '872', '478', '582', '492', '711', '141',
       '475', '181', '733', '682', '465', '949', '911', '474', '493',
       '551', '721', '856', '467', '952', '862', '961', '681', '476',
       '473', '477', '715', '529', '855', '107', '912', '452', '464',
       '471', '463', '651', '222', '953', '969', '479', '423', '424',
       '132', '641', '851', '411', '472', '422', '291', '421', '902',
       '713', '751', '466', '762', '340', '871', '701', '581', '494'],
      dtype=object)

# 가중치 계산

In [65]:
df__test = df_test_1

In [66]:
df__test

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text
90000,id_0251409,C,25,259,철,"프레스가공,용접",자동차부품(외관),"철 프레스가공,용접 자동차부품(외관)"
90001,id_0408083,I,56,562,커피숍에서,접객시설을 갖추고,커피 제공,커피숍에서 접객시설을 갖추고 커피 제공
90002,id_0021595,I,56,561,영업장에서,접객시설을 갖추고,"치킨,호프","영업장에서 접객시설을 갖추고 치킨,호프"
90003,id_0841248,Q,87,872,어린이집에서,군청의인가를받아,영유아보육,어린이집에서 군청의인가를받아 영유아보육
90004,id_0053907,G,47,478,영업장에서,고객대상으로,화장품소매,영업장에서 고객대상으로 화장품소매
...,...,...,...,...,...,...,...,...
99995,id_0147849,I,56,561,매장에서,접객시설을갖추고,칼국수,매장에서 접객시설을갖추고 칼국수
99996,id_0585525,H,49,492,개인택시로,일반인을 대상으로,승객운송서비스,개인택시로 일반인을 대상으로 승객운송서비스
99997,id_0246750,G,47,479,사업장에서,노니직수입,온라인판매,사업장에서 노니직수입 온라인판매
99998,id_0621645,S,96,961,미장원에서,두발미용서비스제공,"파마,컷트","미장원에서 두발미용서비스제공 파마,컷트"


In [67]:
df__test = df__test.reset_index(drop=True)

In [68]:
df__test = df__test[['digit_1','digit_2','digit_3']]
df__test = df__test.reset_index(drop=True)

df__test

,digit_1,digit_2,digit_3
0,C,25,259
1,I,56,562
2,I,56,561
3,Q,87,872
4,G,47,478
...,...,...,...
9995,I,56,561
9996,H,49,492
9997,G,47,479
9998,S,96,961


In [69]:
df__test['test_1'] = df_label1['label']
df__test['test_2'] = df_label2['label']
df__test['test_3'] = df_label3['label']

df__test

,digit_1,digit_2,digit_3,test_1,test_2,test_3
0,C,25,259,C,25,259
1,I,56,562,I,56,562
2,I,56,561,I,56,561
3,Q,87,872,Q,87,872
4,G,47,478,G,47,478
...,...,...,...,...,...,...
9995,I,56,561,I,56,561
9996,H,49,492,H,49,492
9997,G,47,479,G,46,479
9998,S,96,961,S,96,961


In [78]:
df__test

,digit_1,digit_2,digit_3,test_1,test_2,test_3
0,C,25,259,C,25,259
1,I,56,562,I,56,562
2,I,56,561,I,56,561
3,Q,87,872,Q,87,872
4,G,47,478,G,47,478
...,...,...,...,...,...,...
9995,I,56,561,I,56,561
9996,H,49,492,H,49,492
9997,G,47,479,G,46,479
9998,S,96,961,S,96,961


In [70]:
len(df__test[df__test['digit_1']==df__test['test_1']])/len(df__test)

0.9626

In [85]:
df__test['test_2'] = df__test['test_2'].astype(int)
df__test['digit_2'] = df__test['digit_2'].astype(int)

In [86]:
len(df__test[df__test['digit_2']==df__test['test_2']])/len(df__test)

0.8703

In [87]:
df__test['test_3'] = df__test['test_3'].astype(int)
df__test['digit_3'] = df__test['digit_3'].astype(int)

In [88]:
len(df__test[df__test['digit_3']==df__test['test_3']])/len(df__test)

0.7808

In [90]:
(len(df__test[df__test['digit_1']==df__test['test_1']])/len(df__test))*0.1 +(len(df__test[df__test['digit_2']==df__test['test_2']])/len(df__test))*0.2 +(len(df__test[df__test['digit_3']==df__test['test_3']])/len(df__test))*0.7

0.81688